In [1]:
import secrets
from winternitz_utils import hash_chain 

# Naive 4 Bit Winternitz Signatures (**Secure**)

This notebook contains a similar scheme to the previous notebook, however due to the added checksum signatures cannot be forged.

## Generate Key Pair

**Private Key** ($sk_{0} || sk_{1}$): Generate 2 numbers from 256 bits of randomness (or whatever the hash function input length is).

$$sk_0 \overset{{\scriptscriptstyle\$}}{\leftarrow} \mathbb{\{0, 1\}^{256}}$$     
$$sk_1 \overset{{\scriptscriptstyle\$}}{\leftarrow} \mathbb{\{0, 1\}^{256}}$$     

**Public Key** ($pk_{0} || pk_{1}$): Hash the private keys 16 times (as $2^4 = 16$ for 4 bit messages).

$$h(h(h(h(h(h(h(h(h(h(h(h(h(h(h(h(sk_0)))))))))))))))) = h^{16}(sk_0) = pk_0$$
$$h(h(h(h(h(h(h(h(h(h(h(h(h(h(h(h(sk_1)))))))))))))))) = h^{16}(sk_1) = pk_1$$

In [2]:
private_key_0 = secrets.token_bytes(32) # 256 bits (SHA3-256 accepts inputs of 256 bits)
private_key_1 = secrets.token_bytes(32)

public_key_0 = hash_chain(16, private_key_0) 
public_key_1 = hash_chain(16, private_key_1) 

print(f"{private_key_0.hex()=}")
print(f"{private_key_1.hex()=}")
print(f"{public_key_0.hex()=}")
print(f"{public_key_1.hex()=}")

private_key_0.hex()='e42a5e197df2aa53e790e2166475877c63873daef75c925141984bf3799b98e7'
private_key_1.hex()='de6fbb14448635f9daf723b4233b081c232628df119ec8f2d27aa2346ea57b5c'
public_key_0.hex()='8919511bafb0296e4206b9e824fd449f3cefabae049c47c502295dc07e5e498b'
public_key_1.hex()='27dfba4ed5444d529c6c6ed832bb560a871617eb341749fa0cff6bd12ab5c683'


## Sign Message

In order to sign a message we take our first private key ($sk_0$) and hash it $m$ times. We then take our second private key ($sk_1$) and hash it $16 - m$ times. By hashing the two private keys we receive two signature fragments. These fragments can be concatenated to produce a full signature. The following expressions show how to sign the message $m = 5$:

$$h^m(sk_0) = h^5(sk_0) = sig_0^5$$
$$h^{16-m}(sk_1) = h^{16 - 5}(sk_1) = sig_1^5$$ 

The full signature is simply the concatenation of the two signature fragments:

$$sig^5 = sig_0^5 || sig_1^5$$

In [3]:
message = 5 #0b0101
signature_0 = hash_chain(message, private_key_0)
signature_1 = hash_chain(16 - message, private_key_1)

print(f"{signature_0.hex()=}")
print(f"{signature_1.hex()=}")

signature_0.hex()='5430a01f6d75ffffa4b34f7962af2195268456f4f4eee4c373b297d6f047b1d2'
signature_1.hex()='f60e7728022bc0adef15f720873fdf1e97ce781285c58a72950f0973f9aae21b'


## Verify Message

To verify a signed message the verifier must know the public key and the original message. Because there is two signature fragments there are two checks that have to be done. The first signature fragment ($sig^5_0$) is checked by hashing it $16 - m$ times and ensuring that it is the same as the first public key ($pk_0$). The second signature fragment ($sig^5_1$) is checked by hashing it $m$ times and ensuring that it is the same as the second public key ($pk_1$). 

The following equations show how the verifier can check the validity of the signature:

$$h^{16 - m}(sig^5_0) = h^{16 - m}(h^m(sk_0)) = h^{16 - m + m}(sk_0)= h^{16}(sk_0) = pk_0$$ 

$$h^{m}(sig^5_1) = h^{m}(h^{16-m}(sk_1)) = h^{m + 16 - m}(sk_1)= h^{16}(sk_1) = pk_1$$ 

Therefore to check for validity the following must be true:

$$h^{16 - m}(sig^5_0) \equiv pk_0$$

$$h^{m}(sig^5_1) \equiv pk_1$$

In [4]:
print(hash_chain(16 - message, signature_0) == public_key_0)
print(hash_chain(message, signature_1) == public_key_1)

True
True
